## Run/Pass Pre-snap Prediction
### Metric Track

### Overview
As advancements in data collection in the NFL have continued to grow, so has the adoption of data science to make predictions. The following model was built to give NFL defenses an advantage against the San Francisco 49ers by producing a probability that the next offensive play will be a designed run or pass play based on their tendencies based on pre-play line-up and game situation. By using this model, defensive coaches will be able to call an audible that will give them an advantage.

### Data Source
The data was supplied by the NFL and was from weeks 1-6 of the 2023 season. The training data was compressed using gameID and playID and was focused on San Francisco's offensive plays. QB kneels were removed from the dataset. The final training data contained 451 plays and eleven features.

### Target Variable
The target variable is a Boolean feature that has a value of (1) if it is a designed running play or (0) if it was a pass play. To put an emphasis on play calling, QB scrambles do not count as designed runs.

### Model
The model is a random forest classifier that uses 10 features to produce the probability that the play will be a designed run play. Because of the imbalance of run to pass plays (194-257), the model uses class weights to prevent biases since there is an imbalance in the data.

The model performed well with an accuracy of 80.9% and a ROC AUC of 87.9%. True positives were sought due to the costly nature of calling the wrong play when anticipating a run play. Because of this, precision was sought with a value of 0.81 for both classes.

### Feature Selection
The following features were selected based on factors that could influence an offense's play-calling. The definitions are from the NFL's Big Data Bowl Dataset Description. The SHAP value is just a ranking.

offenseFormation (SHAP 16.51%): Formation used by the possession team. Offensive formations are categorical data that have six options: SHOTGUN, SINGLEBACK, I_FORM, EMPTY, PISTOL, and JUMBO. Offensive formations originally had thirteen null values; these nulls were dropped. The data went through OneHotEncoding.

motionSinceLineset (SHAP: 11.20%): Boolean indicating whether the player went in motion after they were initially set at the line on this play. The initial dataset had 1,480 null values; they were filled using a KNN imputer. This feature went through a Binary Encoder before being passed into the training data.

receiverAlignment (SHAP 8.26%): Enumerated as 0x0, 1x0, 1x1, 2x0, 2x1, 2x2, 3x0, 3x1, 3x2. Receiver alignment is categorical data that has six options; the data went through OneHotEncoding.

down_yardsToGo (SHAP 3.97%): This feature was created by taking the down and multiplying it with the yards to go. This allows for a wider range, punishing later downs with further yards to go. This feature went through StandardScaler.

presnap_score_difference (SHAP 3.33%): This feature was created to capture potential play-calling tendencies based on the score difference. If San Francisco is up, the number will be positive; if they are down, the number will be negative. This went through StandardScaler.

shiftSinceLineset (SHAP 2.87%): Boolean indicating whether the player shifted since the lineset; Rule: Each player has their own lineset moment, and whether they shift is based on if they move more than 2.5 yards from where they were at their lineset moment. There were thirteen null values; because of the small amount, the null values were dropped. This feature went through a Binary Encoder before being passed into the training data.

inMotionAtBallSnap (SHAP 1.57%): Boolean indicating whether the player was in motion at the snap; Rule: If a player is moving faster than 0.62 y/s in the window 0.4 seconds prior to the ball snap and has moved at least 1.2 yards in that window. There were 1,104 null values; they were filled using a KNN imputer. This feature went through a Binary Encoder before being passed into the training data.

gameClock_seconds (SHAP 1.45%): From the data feature gameClock, it is the time on the clock of the play (MM:SS). This was converted into seconds and then went through StandardScaler.

yardlineNumber (SHAP 0.94%): Yard line at line-of-scrimmage. This went through StandardScaler.

quarter (SHAP 0.60%): Game quarter, this went through StandardScaler.

We performed a simple correlation to the play result being a run play. As you can see, features that would indicate a run had a higher correlation than features that would indicate a pass.

Correlation with playResult:
playResult                     1.000000                                   
offenseFormation_I_FORM        0.407478                                              
receiverAlignment_2x1          0.384195                                            
presnap_score_difference       0.282033                                            
offenseFormation_SINGLEBACK    0.227010                                            
shiftSinceLineset              0.217943                                            
offenseFormation_PISTOL        0.144669                                            
offenseFormation_JUMBO         0.108878                                            
receiverAlignment_1x1          0.076817                                            
gameClock_seconds              0.063792                                            
receiverAlignment_2x2          0.012063                                            
quarter                       -0.055429                                            
receiverAlignment_4x1         -0.057986                                            
inMotionAtBallSnap            -0.074765                                            
yardlineNumber                -0.086802                                            
receiverAlignment_3x1         -0.172361                                            
receiverAlignment_3x2         -0.270576                                            
offenseFormation_EMPTY        -0.278012                                            
motionSinceLineset            -0.286902                                            
down_yardsToGo                -0.348579                                            
offenseFormation_SHOTGUN      -0.379380  

### Hyperparameters
Using grid search, we were able to fine tune the model to get the best performance focused on the ROC Score. The best hyperparameters were below:

classifier__max_depth: 10                                          
classifier__max_features: 'sqrt'                               
classifier__min_samples_leaf: 1                             
classifier__min_samples_split: 10                   
classifier__n_estimators: 200              

To prevent overfitting we monitored the cross validation scores. The mean cross validation score was 88.5%, which is a strong average. The standard deviation of 1.9% shows us that there is low variability in the model's performance across folds. 

### Model Performance
The model overall performed well with an accuracy score of 80.9% and a ROC AUC of 87.9%. The confusion matrix below will show that the model was able to accurately identify between run and pass plays. Recall was higher at 87% for pass plays compared to 73% for run plays. The f1-score was 84% for pass plays and 77% for run plays. These results allow the defense to have enough confidence in the model where it could give them a competitive advantage.

Classification Report:

              precision    recall  f1-score   support

         0.0       0.81      0.87      0.84        77
         1.0       0.81      0.73      0.77        59


![ROC Curve](roc_curve.png "ROC Curve Analysis")

![Confusion Matrix](confusion_matrix.png "Confusion Matrix")


### Limitations and Assumptions
While the accuracy and ROC AUC are high, they could still be higher. The model would need to be continuously retrained every week to capture any new tendencies the San Francisco 49ers might start employing. As the season goes on, the model will need to start dropping earlier weeks of data from the training set or have a larger emphasis on more recent weeks. The model could be enhanced by including features that account for player personnel, such as whether key players like Christian McCaffrey are in the game, as this could influence play-calling tendencies.

In [2]:
#Appendix

#Import necessary packages
import pandas as pd
import numpy as np
import seaborn as sns
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, learning_curve, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import BinaryEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc
from sklearn.utils.class_weight import compute_class_weight
from sklearn.impute import KNNImputer



# Data Import
plays_columns = ['gameId', 'playId', 'passResult', 'offenseFormation', 'down', 'yardsToGo', 'receiverAlignment', 'yardlineNumber',
                 'gameClock', 'quarter', 'preSnapHomeScore', 'preSnapVisitorScore']
plays_data = pd.read_csv('/Users/lukeellis/Documents/NFL/nfl-big-data-bowl-2025/plays.csv', usecols=plays_columns)
games_data = pd.read_csv('/Users/lukeellis/Documents/NFL/nfl-big-data-bowl-2025/games.csv')
player_play_data = pd.read_csv('/Users/lukeellis/Documents/NFL/nfl-big-data-bowl-2025/player_play.csv', usecols=['nflId', 'gameId', 'playId', 'teamAbbr', 'hadRushAttempt', 'inMotionAtBallSnap', 'shiftSinceLineset', 'motionSinceLineset'])
players_data = pd.read_csv('/Users/lukeellis/Documents/NFL/nfl-big-data-bowl-2025/players.csv')
teams_data = pd.read_csv('/Users/lukeellis/Documents/NFL/nfl-big-data-bowl-2025/games.csv', usecols=['gameId', 'homeTeamAbbr', 'visitorTeamAbbr'])

# Data Merging
player_play_merged = player_play_data.merge(players_data[['nflId', 'position']], on='nflId', how='left')
play_game_merged = player_play_merged.merge(plays_data, on=['gameId', 'playId'], how='left')
master_data = play_game_merged.merge(teams_data, on='gameId', how='left')

# Data Filtering and Preprocessing
master_data = master_data[(master_data['teamAbbr'] == 'SF') & (master_data['position'].isin(['QB', 'RB', 'WR', 'TE', 'FB']))]
master_data['presnap_score_difference'] = np.where(master_data['homeTeamAbbr'] == 'SF', 
                                                   master_data['preSnapHomeScore'] - master_data['preSnapVisitorScore'],
                                                   master_data['preSnapVisitorScore'] - master_data['preSnapHomeScore'])

master_data = master_data[master_data['offenseFormation'] != 'None'].dropna(subset=['shiftSinceLineset', 'offenseFormation'])
#Create target value
master_data['playResult'] = master_data['passResult'].apply(lambda x: 1 if pd.isna(x) else 0)

# Data Aggregation
data_aggregated = master_data.groupby(['gameId', 'playId'], as_index=False).agg({
    'hadRushAttempt': 'max',
    'inMotionAtBallSnap': 'max',
    'shiftSinceLineset': 'max',
    'motionSinceLineset': 'max',
    'offenseFormation': 'first',
    'playResult': 'first',
    'down': 'first',
    'passResult': 'first',     
    'yardsToGo': 'first',
    'receiverAlignment': 'first',
    'gameClock': 'first',
    'yardlineNumber': 'first',
    'quarter': 'first',
    'presnap_score_difference': 'first'
})
data_aggregated['down_yardsToGo'] = data_aggregated['down'] * data_aggregated['yardsToGo']

#Drop unnecessary columns
columns_to_drop = ['gameId', 'playId', 'hadRushAttempt','passResult', 'down', 'yardsToGo']
data_aggregated = data_aggregated.drop(columns=columns_to_drop)

#Convert feature to seconds
def time_to_seconds(time_str):
    if pd.isna(time_str):
        return np.nan
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds


data_aggregated['gameClock_seconds'] = data_aggregated['gameClock'].apply(time_to_seconds)
data_aggregated = data_aggregated.drop('gameClock', axis=1)

# Encoding and Feature Selection
categorical_features = ['offenseFormation', 'receiverAlignment']
data_encoded = pd.get_dummies(data_aggregated, columns=categorical_features)

# Correlation Analysis
corr = data_encoded.corr()['playResult'].sort_values(ascending=False)
print("Correlation with playResult:")
print(corr)


# KNN imputate to fix remaining null values
def knn_impute(df, target_column, n_neighbors=5):
    numerical_df = df.select_dtypes(include=['float64', 'int64', 'bool'])
    knn_imputer = KNNImputer(n_neighbors=n_neighbors)
    imputed_data = knn_imputer.fit_transform(numerical_df)
    imputed_df = pd.DataFrame(imputed_data, columns=numerical_df.columns)
    df[numerical_df.columns] = imputed_df
    return df

data_aggregated = knn_impute(data_aggregated, 'inMotionAtBallSnap')
data_aggregated = knn_impute(data_aggregated, 'motionSinceLineset')

# Model Preparation
X = data_aggregated.drop('playResult', axis=1)
y = data_aggregated['playResult']


# Seperate features
numeric_features = ['down_yardsToGo', 'yardlineNumber', 'gameClock_seconds', 'presnap_score_difference', 'quarter']
categorical_features = ['offenseFormation', 'receiverAlignment']
binary_features = ['shiftSinceLineset', 'motionSinceLineset', 'inMotionAtBallSnap']

# Preprocessing Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features),
        ('motion', BinaryEncoder(), binary_features)
    ],
    remainder='passthrough'
)

# Model Pipeline
class_weights = compute_class_weight('balanced', classes=[0, 1], y=y)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(class_weight=class_weights_dict, random_state=42))
])

# Hyperparameter Tuning
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__max_features': ['sqrt', 'log2', None]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc', n_jobs=-1, verbose=1)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train model
print("Training model...")
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
print("\nBest parameters:", grid_search.best_params_)

# Predictions
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
y_pred = best_model.predict(X_test)

# Metrics
metrics = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'ROC AUC': roc_auc_score(y_test, y_pred_proba)
}

#Using Stratified Cross Validation to prevent overfitting
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(best_model, X, y, cv=cv, scoring='roc_auc')

metrics['CV Scores'] = cv_scores

print("\nModel Evaluation Metrics:")
for metric, value in metrics.items():
    if metric != 'CV Scores':
        print(f"{metric}: {value:.3f}")
    else:
        print(f"{metric}: {value}")
        print(f"Mean CV score: {value.mean():.3f}")
        print(f"Standard Deviation of CV scores: {value.std():.3f}")

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# ROC Curve
plt.figure(figsize=(5, 3))
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.savefig('roc_curve.png', dpi=300, bbox_inches='tight')
plt.close()



# List of base features
base_features = [
    'shiftSinceLineset', 'motionSinceLineset', 'inMotionAtBallSnap',
    'offenseFormation', 'receiverAlignment', 'down_yardsToGo', 'quarter',
    'yardlineNumber', 'gameClock_seconds', 'presnap_score_difference'
]
feature_names = best_model.named_steps['preprocessor'].get_feature_names_out().tolist()


# Transform X_test using the preprocessor
X_test_transformed = best_model.named_steps['preprocessor'].transform(X_test)

# SHAP Explainer
explainer = shap.TreeExplainer(best_model.named_steps['classifier'])  # Adjust 'classifier' step if different
shap_values = explainer.shap_values(X_test_transformed)

# Aggregate SHAP values across the class dimension (e.g., take mean absolute values)
shap_values = np.abs(shap_values).mean(axis=2)  # Shape becomes (n_samples, n_features)

# Calculate mean absolute SHAP values across samples
mean_abs_shap_values = shap_values.mean(axis=0)  # Shape becomes (n_features,)

# Verify alignment with feature names
if len(mean_abs_shap_values) != len(feature_names):
    raise ValueError(f"Shape mismatch: {len(mean_abs_shap_values)} SHAP values vs. {len(feature_names)} feature names.")

# Map each feature name to its base category
def map_to_base_feature(feature_name):
    for base_feature in base_features:
        if base_feature in feature_name:
            return base_feature
    return feature_name  # If no match, return the original name

mapped_features = [map_to_base_feature(name) for name in feature_names]

# Group SHAP values by the base feature
shap_df = pd.DataFrame({'Feature': mapped_features, 'SHAP Value': mean_abs_shap_values})
grouped_shap = shap_df.groupby('Feature')['SHAP Value'].sum().sort_values(ascending=False)

# Print grouped feature ranking
print("\nFeature Ranking Based on SHAP Values (Grouped):")
for rank, (feature_name, shap_value) in enumerate(grouped_shap.items(), 1):
    print(f"{rank}. {feature_name} ({shap_value:.4f})")

#Create Confusion Matrix

conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1, keepdims=True)

# Plot the heatmap
plt.figure(figsize=(8, 6))
ax = sns.heatmap(conf_matrix_normalized, annot=False, fmt='.2%', cmap='Blues',
                 xticklabels=['Predicted Run', 'Predicted Pass'], 
                 yticklabels=['Actual Run', 'Actual Pass'])

# Titles and labels
plt.title('Confusion Matrix', fontsize=14)

# Annotate each cell with count and percentage
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        count = conf_matrix[i, j]
        percentage = conf_matrix_normalized[i, j]
        text_color = "white" if (i == 0 and j == 0) else "black"  # White text for upper-left cell only
        ax.text(j + 0.5, i + 0.5, f"{count}\n({percentage:.2%})",
                ha="center", va="center", color=text_color, fontsize=10)

# Save and show plot
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.close()


Correlation with playResult:
playResult                     1.000000
offenseFormation_I_FORM        0.407478
receiverAlignment_2x1          0.384195
presnap_score_difference       0.282033
offenseFormation_SINGLEBACK    0.227010
shiftSinceLineset              0.217943
offenseFormation_PISTOL        0.144669
offenseFormation_JUMBO         0.108878
receiverAlignment_1x1          0.076817
gameClock_seconds              0.063792
receiverAlignment_2x2          0.012063
quarter                       -0.055429
receiverAlignment_4x1         -0.057986
inMotionAtBallSnap            -0.074765
yardlineNumber                -0.086802
receiverAlignment_3x1         -0.172361
receiverAlignment_3x2         -0.270576
offenseFormation_EMPTY        -0.278012
motionSinceLineset            -0.286902
down_yardsToGo                -0.348579
offenseFormation_SHOTGUN      -0.379380
Name: playResult, dtype: float64
Training model...
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

Best parameters